# Part 1: Black Box KLVI 

This notebook is the first of a three part series, which walks through the key results of my variational inference thesis. The intention is to allow the user to walk through my thesis results, step by step.

**Thesis Problem: Use Alternative Divergence Measures to solve Variance Underestimation for a single factor Confirmatory Factor Analysis Model** 

My thesis  investigates whether we can use a generalised divergence measure to perform black box variational inference to solve the problem of variance underestimation. 

The problem of variance underestimation is when the estimated posteriors underestimate the true posterior variance. It occurs whenever we use the typical divergence measure (KL Divergence), in both analytical **and** black-box implementations of VI.

**Notebook Purpose**

This notebook focuses on applying black box KL Variational Inference (KLVI) to the single factor confirmatory factor analysis model (CFA model), using the classic Holzinger Swineford (1939) dataset.

The notebook has two outcomes:

* **I demonstrate that black box KLVI can be applied to the single CFA model.** 
    * Effort is made to demonstrate how to tune optimisation parameters.

* **I replicate the finding of variance underestimation via black box KLVI** 
    * We replicate the results of KD Dang, who applied KLVI to the single CFA model via analytical methods rather than black box methods. Dang found that analytical KLVI estimates posterior means accurately but underestimates true posterior variances. We find the same results hold for black box KLVI.

# Imports 

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from semopy.examples import holzinger39
import torch
import pickle

In [14]:
import sys 
sys.path.append('..')
from src.statistical_model import SingleFactorCFA
from src.variational_family import SingleCFAVariationalFamily
from src.variational_sem import SemVIModel, VIOptimisationParameters
from src.analytical_variational_inference.analytical_variational_infernce import single_factor_cfa_mfvb
from src.analysis.sampling import create_sample_from_qvar, SingleCFAVariationalParameters
from src.mcmc.mcmc import single_factor_cfa_mcmc
from torch.distributions import MultivariateNormal as mvn

# Introduction

# Single Factor CFA - Model Definition

## Dataset

We are interested in the first three test results from the Holzinger Swineford (1939) dataset, which is the test results from visual perception, cubes, and lozenges test results.

* Extract dataset 
* Convert to Pytorch Tensor 
* De-mean 

In [4]:
hdata = holzinger39.get_data()

In [8]:
hdata.head()

,id,sex,ageyr,agemo,school,grade,x1,x2,x3,x4,x5,x6,x7,x8,x9
1,1,1,13,1,Pasteur,7.0,3.333333,7.75,0.375,2.333333,5.75,1.285714,3.391304,5.75,6.361111
2,2,2,13,7,Pasteur,7.0,5.333333,5.25,2.125,1.666667,3.00,1.285714,3.782609,6.25,7.916667
3,3,2,13,1,Pasteur,7.0,4.500000,5.25,1.875,1.000000,1.75,0.428571,3.260870,3.90,4.416667
4,4,1,13,2,Pasteur,7.0,5.333333,7.75,3.000,2.666667,4.50,2.428571,3.000000,5.30,4.861111
5,5,2,12,2,Pasteur,7.0,4.833333,4.75,0.875,2.666667,4.00,2.571429,3.695652,6.30,5.916667


In [5]:
y_data = hdata[['x1', 'x2', 'x3']]

Since we are utilising pytorch to perform black box variational inference, we will deal exclusively with Pytorch tensors.

In [10]:
#No gradients needed on y_data
y_data = torch.tensor(y_data.values, requires_grad = False)

# Single Factor CFA Model  

We use the same statistical model as that used by Dang and Maestrini, including the same dataset and prior hyperparameter values. 

## Prior Hyperparameters 



In [6]:
#Set Hyper-parameters 
#sig_2 ~ InvGamma
sig2_shape = torch.tensor([0.5])  
sig2_rate = torch.tensor([0.5])  

#psi ~ iid Inv Gamma for j = 1..m 
psi_shape = torch.tensor([0.5])  
psi_rate = torch.tensor([0.005])  

#nu ~ iid Normal for j = 1...m
nu_sig2 = torch.tensor([100.0])  
nu_mean = torch.tensor([0.0])

#lam_j | psi_j ~ id Normal(mu, sig2*psi_j)
lam_mean = torch.tensor([0.0])
lam_sig2 = torch.tensor([1.0])

hyper_params = {"sig2_shape": sig2_shape, "sig2_rate": sig2_rate, "psi_shape": psi_shape, "psi_rate": psi_rate, "nu_sig2": nu_sig2, "nu_mean": nu_mean, "lam_mean": lam_mean, "lam_sig2": lam_sig2}


In [12]:
single_cfa_model = SingleFactorCFA(y_data = y_data, hyper_params= hyper_params)

## Mean Field Variational Family

In [13]:
single_cfa_qvar = SingleCFAVariationalFamily(m = y_data.shape[1], n = y_data.shape[0])

## VI Model

In [14]:
vi_model = SemVIModel(single_cfa_model, single_cfa_qvar)

# Black Box KLVI

In [15]:
VI_OPTIMISATION_PARAMETERS = VIOptimisationParameters(num_iterations = 100, relative_error_threshold= 10e-4, patience = 100)

In [70]:
vi_model.optimize(K = 100, alpha = 1, optimisation_parameters= VI_OPTIMISATION_PARAMETERS, filename = 'tensorboard_runs/OptimiseVIModel')

100%|██████████| 100/100 [00:45<00:00,  2.18it/s]


## Visualising Convergence

In [73]:
%load_ext tensorboard
%tensorboard --logdir tensorboard_runs/OptimiseVIModel_K_100_alpha1/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 12776), started 0:01:00 ago. (Use '!kill 12776' to kill it.)

## Results Comparison to Variational Bayes (Mean Field) and MCMC 

In [ ]:
#TODO: Get MCMC code running
#mcmc_sample = single_factor_cfa_mcmc(y_data, hyperparams)

#Step 1: Load in Pickled MCMC Data  
#Step 2: Implement MFVB code (DONE)
#Step 3: Visualise results 
#Load in Pickled KLVI dataframe K = 1, K = 10, K = 100 
#Resurrect visualisations - 95% credible interval and mean 

## MCMC

## Mean Field Variational Bayes 

In [22]:
# Mean Field Variational Bayes 
mfvb_model = single_factor_cfa_mfvb()

lavaan is FREE software! Please report any bugs.
  


Iteration number: 2 ; MFVB relative error: 255.02 
Iteration number: 3 ; MFVB relative error: 230.4 
Iteration number: 4 ; MFVB relative error: 144.34 
Iteration number: 5 ; MFVB relative error: 12.525 
Iteration number: 6 ; MFVB relative error: 25.803 
Iteration number: 7 ; MFVB relative error: 3.013 
Iteration number: 8 ; MFVB relative error: 6.5939 
Iteration number: 9 ; MFVB relative error: 2.6136 
Iteration number: 10 ; MFVB relative error: 30.616 
Iteration number: 11 ; MFVB relative error: 2.1981 
Iteration number: 12 ; MFVB relative error: 8.2253 
Iteration number: 13 ; MFVB relative error: 1.9037 
Iteration number: 14 ; MFVB relative error: 3.6124 
Iteration number: 15 ; MFVB relative error: 0.72829 
Iteration number: 16 ; MFVB relative error: 0.61567 
Iteration number: 17 ; MFVB relative error: 1.4817 
Iteration number: 18 ; MFVB relative error: 90.845 
Iteration number: 19 ; MFVB relative error: 1.0416 
Iteration number: 20 ; MFVB relative error: 22.978 
Iteration number: 21

In [23]:
mfvb_model

SingleCFAVariationalParameters(nu=MultivariateNormal(loc: torch.Size([3]), covariance_matrix: torch.Size([3, 3])), lam=MultivariateNormal(loc: torch.Size([2]), covariance_matrix: torch.Size([2, 2])), psi=<src.pdfs.InverseGamma object at 0x28f3fd750>, sig2=<src.pdfs.InverseGamma object at 0x28f3fce50>)

In [24]:
mfvb_sample = create_sample_from_qvar(mfvb_model)

In [26]:
mfvb_sample.mean()

nu.1     4.935342
nu.2     6.087354
nu.3     2.249882
lam.1    0.751374
lam.2    1.038437
psi.1    0.813815
psi.2    1.073257
psi.3    0.677031
sig2     0.560578
dtype: float64

In [ ]:
{'MCMC': mcmc_sample, 
 'MFVB': mfvb_sample, 
}

# Optimisation & Parameter Tuning 

# What is the optimal K? 

# Note about convergence time 

# Testing Procedure 


# Test 1: Use mock simulated data

The most high level test to assess whether our code is working as intended is to simulate some y_data from known distributions and impose degenerate priors.

In [31]:
N = 300
M = 3

#Generate Non-Latent Parameters
nu = torch.tensor([5.0, 6.0, 2.25])
sig2= torch.tensor(0.565)
lam = torch.tensor([0.75, 1.05])
psi = torch.tensor([0.8, 1.10, 0.66])

#Generate Latent Parameter Values
eta = torch.randn(N)*torch.sqrt(sig2)

lam_1_fixed = torch.tensor([1.0])
lam_full = torch.cat((lam_1_fixed, lam))

#Generate y values based on User Input
# yi ~ id Normal(nu + eta_i * lam, diag(psi)), yi /in R^m
#cov:
like_dist_cov = torch.diag(psi) #m*m tensor 
#means: want a n*m vector of means
like_dist_means = torch.matmul(eta.unsqueeze(1), lam_full.unsqueeze(0)) + nu

y_data_sim = mvn(like_dist_means, covariance_matrix= like_dist_cov).rsample(torch.tensor([10])) 

In [ ]:
degenerate_priors = {
    
}

In [ ]:
simulated_cfa_model = SingleFactorCFA(y_data = y_data_sim, hyper_params=hyper_params)

In [15]:
simulated_vi_model = SemVIModel(model = simulated_cfa_model, qvar = SingleCFAVariationalFamily(m = M, n = N))

In [21]:
VI_OPTIMISATION_PARAMETERS = VIOptimisationParameters(num_iterations = 20000, relative_error_threshold= 10e-4, patience = 100)

In [22]:
simulated_vi_model.optimize(optimisation_parameters = VI_OPTIMISATION_PARAMETERS, K = 10, alpha = 1, filename = 'tensorboard_runs/simulatedVIModel')

 32%|███▏      | 6487/20000 [04:51<10:07, 22.25it/s]

VI converged at step t =  6487


In [23]:
%reload_ext tensorboard
%tensorboard --logdir tensorboard_runs/simulatedVIModel_K_10_alpha1/

Reusing TensorBoard on port 6006 (pid 53556), started 0:06:56 ago. (Use '!kill 53556' to kill it.)

In [25]:
(85.82)/(152.4)

0.5631233595800524

In [27]:
895.1/(1116)

0.8020609318996416

In [29]:
(1172)/(1091)

1.074243813015582

In [30]:
823.6/1217

0.6767460969597371

In [32]:
qvar = SingleCFAVariationalFamily(m = M, n = N)